In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
import time



In [49]:
# Load the trained CNN model
#model = tf.keras.models.load_model('model.h5')
BS=32
INIT_LR = 1e-4
EPOCHS = 10
# model = tf.keras.models.load_model('../Downloads/CloseOrOpenEye-2.model', compile=False)
model = tf.keras.models.load_model('../Downloads/model-5.h5', compile=False)
imagemodel = tf.keras.models.load_model('../Downloads/CloseOrOpenEye-2.model', compile=False)

# Define the optimizer
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

# Compile the model with the specified optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
imagemodel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [50]:
# Define the emotion labels
emotion_labels = ['happiness',  'sadness', 'neutral']

In [51]:
face_cascade = cv2.CascadeClassifier('/Users/siddharthpatel/Downloads/archive-3/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/Users/siddharthpatel/Downloads/archive-3/haarcascade_eye.xml')

In [55]:
BS=32
flag = 0
flag_eye=0
starttime = time.time()
starttime_eye = time.time()
# Start the video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 10)

# Define the threshold
threshold = 0.5
window_size_secs = 30
emotion_buffer = deque(maxlen=int(window_size_secs * 10))
window_buffer = deque(maxlen=int(window_size_secs * 10))
total_emotions = deque(maxlen=int(window_size_secs * 10))
eye_buffer = deque(maxlen=int(window_size_secs * 10))
eye_window_buffer = deque(maxlen=int(window_size_secs * 10))
total_eye_emotions = deque(maxlen=int(window_size_secs * 10))

In [56]:
while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Preprocess the frame
    eyeimage = frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48), interpolation=cv2.INTER_AREA)
    img = np.reshape(resized, (1, 48, 48, 1)) / 255.0

    # Pass the frame through the CNN model
    pred = model.predict(img)

    label_prob = np.max(pred)
    if label_prob < threshold:
        label = 2  # set the label to 'neutral'
    else:
        label = np.argmax(pred)
    text = emotion_labels[label]
    emotion_buffer.append(label)
    window_buffer.append(label)

    # Post-process the results
    label = np.argmax(pred)
    text = emotion_labels[label]
    
    if len(window_buffer) == int(window_size_secs * 10):
        
        # Calculate the most common emotion in the window buffer
        most_common = max(set(window_buffer), key=window_buffer.count)
#         text = emotion_labels[most_common]
        total_emotions.append(most_common)
#         if flag == 0 and time.time() - starttime > 30:
        time.time() - starttime > 30:
            if total_emotions == "sadness":
                start_playback()
                flag = 1
                starttime = time.time()
                continue
            elif total_emotions != "sadness":
                stop_playback()
                flag = 0
                
            else:
                flag = 0
                continue

        # Remove the oldest data point from the window buffer and emotion buffer
        window_buffer.popleft()
        emotion_buffer.popleft()
    gray = cv2.cvtColor(eyeimage, cv2.IMREAD_GRAYSCALE)#converting the input image to greay image
    face_cascade = cv2.CascadeClassifier('/Users/siddharthpatel/Downloads/archive-3/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('/Users/siddharthpatel/Downloads/archive-3/haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(eyeimage,scaleFactor=1.1, minNeighbors=4)
    # for x,y,w,h in eyes:
    #     cv2.rectangle(eyeimage,(x,y),(x+w ,y+h), (0, 255, 0), 2)
    # plt.imshow(cv2.cvtColor(eyeimage, cv2.COLOR_BGR2RGB))
    # plt.show()
    
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = eyeimage[y:y+h, x:x+w]
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("eyes are not detected")
            
        else:
            for(ex,ey,ew,eh) in eyess:
                eyes_roi = roi_color[ey: ey+eh, ex:ex+ew]
    eyes_roi.shape
    final_image = cv2.resize(eyes_roi,(224,224))
    #final_image = eyeimage_to_array(final_image)
    #final_image = np.reshape(224,224)
    final_image = np.expand_dims(final_image, axis = 0)

    final_image = preprocess_input(final_image)

    prediction = imagemodel.predict(final_image,BS)
    print(prediction)
    #i = 0
    #predIdxs = model.predict(image, batch_size=BS)
    predIdxs = np.argmax(prediction, axis=1)
    print(predIdxs)
    if predIdxs[0] == 0 :
        prediction_class = 'Closed'
    else :
        prediction_class = 'Open'

    print(prediction_class)
    eye_buffer.append(prediction_class)
    eye_window_buffer.append(prediction_class)
    
    if len(window_buffer) == int(window_size_secs * 10):
        # Calculate the most common emotion in the window buffer
        most_common_eye = max(set(eye_window_buffer), key=eye_window_buffer.count)
#         text = emotion_labels[most_common]
        total_eye_emotions.append(most_common_eye)
#         if flag == 0 and time.time() - starttime > 30:
        time.time() - starttime > 2:
            if total_emotions == "sadness":
                start_playback()
                flag_eye = 1
                starttime_eye = time.time()
                continue
            elif total_emotions != "sadness":
                stop_playback()
                flag_eye = 0
                continue
                
            else:
                flag_eye = 0
                continue
    
    # Display the results on the video
    cv2.putText(frame, text+" "+prediction_class, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    cv2.imshow('frame', frame)
    # Wait for a key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 0s 38ms/step
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 414ms/step
[[0.18834253 0.64327884]]
[1]
Open
1/1 [==============================] - 0s 77ms/step
eyes are not detected
1/1 [==============================] - 0s 56ms/step
[[9.9999994e-01 1.0115929e-07]]
[0]
Closed
1/1 [==============================] - 0s 29ms/step
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 37ms/step
[[9.999995e-01 6.146128e-07]]
[0]
Closed
1/1 [==============================] - 0s 29ms/step
[[9.9998134e-01 1.9864277e-05]]
[0]
Closed
1/1 [==============================] - 0s 40ms/step
eyes are not detected
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 30ms/step
[[9.9998134e-01 1.9864277e-05]]
[0]
Closed
1/1 [==============================] - 0s 26ms/step
eyes are not detected
eyes are not detected
1/1 [=====

1/1 [==============================] - 0s 22ms/step
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 29ms/step
[[9.9996221e-01 2.1297432e-05]]
[0]
Closed
1/1 [==============================] - 0s 23ms/step
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[9.9995917e-01 3.1210304e-05]]
[0]
Closed
1/1 [==============================] - 0s 23ms/step
eyes are not detected
1/1 [==============================] - 0s 27ms/step
[[0.6214242  0.12268828]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.9916798e-01 2.7286442e-04]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 27ms/step
[[9.9916798e-01 2.7286442e-04]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[9.9916798e-01 2.728

1/1 [==============================] - 0s 23ms/step
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[0.98212224 0.01611255]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[0.98212224 0.01611255]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[0.98212224 0.01611255]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.9999648e-01 3.0877861e-06]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.9996424e-01 6.3031614e-05]]
[0]
Closed
1/1 [==============================] - 0s 25ms/step
eyes are not detected
1/1 [==============================] - 0s 32ms/step
[[9.9996424e-01 6.3031614e-05]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 29ms/step
[[9.9996424e-01 6.3031614e

eyes are not detected
1/1 [==============================] - 0s 29ms/step
[[9.999948e-01 3.567241e-06]]
[0]
Closed
1/1 [==============================] - 0s 23ms/step
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[9.999948e-01 3.567241e-06]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.9999875e-01 2.1841415e-06]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.9998003e-01 4.6310888e-06]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[0.9911334  0.00451813]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.9943155e-01 1.8892586e-04]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[0.9772905  0.01297102]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.998077e-01 8.130691e-05]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 27ms/step
[[9.9998486e-01 9.1189277e-06]]
[0]


1/1 [==============================] - 0s 27ms/step
[[9.9999988e-01 1.0530163e-07]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.9999869e-01 2.2696686e-06]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.999948e-01 4.573392e-06]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 28ms/step
[[9.9999851e-01 2.2316995e-06]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.9999553e-01 2.9324403e-06]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.999992e-01 7.229549e-07]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.9999970e-01 2.0518277e-07]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.9999970e-01 2.0518277e-07]]
[0]
Closed
1/1 [==============================] - 0s 32ms/step
[[9.995387e-01 2.079259e-04]]
[0]
Closed
1/1 [==============================] - 0s 23ms/step
eyes are not detected
1/1

1/1 [==============================] - 0s 29ms/step
[[0.99386525 0.00334928]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.9958354e-01 2.0373252e-04]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[7.7031006e-04 9.9869657e-01]]
[1]
Open
1/1 [==============================] - 0s 27ms/step
[[0.9188492  0.02403449]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[0.7418806  0.16344759]]
[0]
Closed
1/1 [==============================] - 0s 32ms/step
[[0.5906118  0.34428433]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[0.5906118  0.34428433]]
[0]
Closed
1/1 [==============================] - 0s 22ms/step
eyes are not detected
1/1 [==============================] - 0s 27ms/step
[[9.9998826e-01 9.1874417e-06]]
[0]
Closed
1/1 [==============================] - 0s 27ms/step
[[9.999965e-01 4.741843e-06]]
[0]
Closed
1/1 [==============================] - 0s 28ms/step
[[9.999965e-01 4.700677e-06]]
[0]
Closed
1/1 [=======

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# import vlc
import threading
import random

# Authenticate and authorize access to the Spotify Web API
client_credentials_manager = SpotifyClientCredentials(client_id='0a642fbb2af4493cb8aa8a929479f180', client_secret='90cd2106dcfb4a20a0530db64bceaa70')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Use the Spotify recommendation API to get a list of recommended tracks based on the user's current mood
recommendations = sp.recommendations(seed_genres=[current_mood.lower()], limit=50)
mood = "sadness"
# Filter the recommended tracks to find tracks with a high danceability score
high_danceability_tracks = []
for track in recommendations['tracks']:
    if track['preview_url'] != None and track['preview_url'] != "None":
        high_danceability_tracks.append(track)

# Display the list of tracks
print(f"Here are the top {len(recommendations['tracks'])} tracks for '{mood}':")
for i, track in enumerate(recommendations['tracks']):
    track_name = track['name']
    track_artist = track['artists'][0]['name']
    print(f"{i+1}. {track_name} by {track_artist}  {track['preview_url']}")
chosen_track_index=random.randint(1, 50)
# Prompt the user to choose a track
# chosen_track_index = int(input(f"Enter the number of the track you want to play (1-{len(recommendations['tracks'])}): ")) - 1
chosen_track = recommendations['tracks'][chosen_track_index]

# Retrieve the track data
track_name = chosen_track['name']
track_artist = chosen_track['artists'][0]['name']
track_preview_url = chosen_track['preview_url']

# Create a new instance of the VLC media player
vlc_instance = vlc.Instance('--no-xlib')

def start_playback():
    player.play()

# Load the track into the player
media = vlc_instance.media_new(track_preview_url)

# Play the track in the VLC media player
player = vlc_instance.media_player_new()
player.set_media(media)
start_playback()

# Define a function to stop the playback and exit the program
def stop_playback():
    player.stop()
    vlc_instance.release()

# Start a thread to wait for user input
input_thread = threading.Thread(target=input, args=('Press "q" to stop playback\n',))
input_thread.daemon = True
input_thread.start()

# Wait for the track to finish playing or for the user to interrupt the program
while player.get_state() != vlc.State.Ended:
    if input_thread.is_alive():
        input_thread.join(timeout=0.1)
    else:
        break

# Stop the playback and exit the program
stop_playback()


In [15]:
!pip install spotipy
!pip show spotipy
import sys
sys.path.append('/Users/siddharthpatel/ENTER/lib/python3.10/site-packages')
import spotipy
!pip install python-vlc


Name: spotipy
Version: 2.22.1
Summary: A light weight Python library for the Spotify Web API
Home-page: https://spotipy.readthedocs.org/
Author: @plamere
Author-email: paul@echonest.com
License: MIT
Location: /Users/siddharthpatel/ENTER/lib/python3.10/site-packages
Requires: redis, requests, six, urllib3
Required-by: 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [ ]:
# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()